<a href="https://colab.research.google.com/github/nikolmash/yes-no-question-answering/blob/master/QA_bert_feature_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers
!pip install jsonlines
import numpy as np
import pandas as pd
import tensorflow as tf 
import jsonlines
from transformers import *
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

     |████████████████████████████████| 665kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 12.8MB/s 
     |████████████████████████████████| 1.1MB 44.6MB/s 
     |████████████████████████████████| 890kB 41.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=105b1c73e85d00d6c825826ad4a8e482bef29ca1d76db22070a6b8d2870daf9a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with jsonlines.open('/content/drive/My Drive/train.jsonl') as reader:
      train = [obj for obj in reader]
with jsonlines.open('/content/drive/My Drive/dev.jsonl') as reader:
      dev = [obj for obj in reader]

In [0]:
train_df = pd.DataFrame(train)
dev_df = pd.DataFrame(dev)

In [0]:
bert_model = TFBertModel.from_pretrained("bert-base-uncased") 
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [0]:
train_df['question'] = train_df['question'].apply(bert_tokenizer.tokenize)
train_df['passage'] = train_df['passage'].apply(bert_tokenizer.tokenize)
dev_df['question'] = dev_df['question'].apply(bert_tokenizer.tokenize)
dev_df['passage'] = dev_df['passage'].apply(bert_tokenizer.tokenize)

In [0]:
max_length = 512
train_input_ids = []
train_token_ids = []

for question, text in zip(train_df['question'].values, train_df['passage'].values):
  encoded_data = bert_tokenizer.encode_plus(question, text, max_length=max_length, add_special_tokens=True,
                                            pad_to_max_length=True, truncation_strategy="longest_first")
  input_ids, token_type_ids = encoded_data['input_ids'], encoded_data['token_type_ids']
  train_input_ids.append(input_ids)
  train_token_ids.append(token_type_ids)

In [0]:
dev_input_ids = []
dev_token_ids = []

for question, text in zip(dev_df['question'].values, dev_df['passage'].values):
  encoded_data = bert_tokenizer.encode_plus(question, text, max_length=max_length, add_special_tokens=True,
                                            pad_to_max_length=True, truncation_strategy="longest_first")
  input_ids, token_type_ids = encoded_data['input_ids'], encoded_data['token_type_ids']
  dev_input_ids.append(input_ids)
  dev_token_ids.append(token_type_ids)

In [0]:
train_answers = train_df['answer'].values.astype(int)
dev_answers = dev_df['answer'].values.astype(int)

In [0]:
train_embeddings = []
for i in tqdm(range(len(train_input_ids))):
  output = bert_model(tf.constant(train_input_ids[i])[None, :], token_type_ids=tf.constant(train_token_ids[i])[None, :])
  last_4_hid_layers = output[0][:,:,-5:-1].numpy()
  train_embeddings.append(last_4_hid_layers.mean(axis=2))

In [15]:
dev_embeddings = []
for i in tqdm(range(len(dev_input_ids))):
  output = bert_model(tf.constant(dev_input_ids[i])[None, :], token_type_ids=tf.constant(dev_token_ids[i])[None, :])
  last_4_hid_layers = output[0][:,:,-5:-1].numpy()
  dev_embeddings.append(last_4_hid_layers.mean(axis=2))


In [0]:
for i in range(len(train_embeddings)):
  train_embeddings[i] = train_embeddings[i].reshape(512,)
for i in range(len(dev_embeddings)):
  dev_embeddings[i] = dev_embeddings[i].reshape(512,)

In [0]:
lr = LogisticRegression(max_iter=1000)
lr.fit(train_embeddings, train_answers)
predictions = lr.predict(dev_embeddings)

In [25]:
print('Accuracy: ', accuracy_score(dev_answers, predictions))
print('F1 score: ', f1_score(dev_answers, predictions))

Accuracy:  0.6137614678899083
F1 score:  0.7459263729631865
